In [2]:
import os
import re
import numpy as np

In [4]:
"""
Function to read the warsaw parameters from the file given by Rosetta after the translation from SILH
"""
def read_warsaw_parameters(warsaw_file_in):
    parameters = {}
    pattern = re.compile(r'([+-]?\d+\.\d+e[+-]\d+)\s+#\s+(\w+)')
    
    with open(warsaw_file_in, 'r') as file:
        for line in file:
            match = pattern.search(line)
            if match:
                value, param_name = match.groups()
                parameters[param_name] = value
                # Print found parameters for debugging
                print(f"Found in warsaw file: {param_name} = {value}")
    
    return parameters

In [6]:
"""
Function to replace the values of the WCs in the param_card of SMEFTsim UFO file

restric_file_in : A .dat file which is in the same format as the input card for SMEFTsim UFO. with all WCs=0
parameters : the parameters read from the .dat file from Rosetta (output of the function read_warsaw_parameters)
mapping : dictionary which maps the variable names from the Rosetta file to the SMEFTsim friendly input card.
"""
def update_restrict_file(restrict_file_in, parameters, mapping):
    """Update parameters in restrict_all_zero.dat file based on mapping while preserving formatting."""
    with open(restrict_file_in, 'r') as file:
        lines = file.readlines()
    
    # Updated pattern to match parameter lines in restrict file with index at the beginning
    # Format: "152 0.000000e+00 # cHudRe22"
    pattern = re.compile(r'(\s*)(\d+)(\s+)([+-]?\d+\.\d+e[+-]\d+)(\s+#\s+)(\w+)(.*)')
    updated_lines = []
    
    for line in lines:
        line = line.rstrip('\n')  # Remove newline but will add it back later
        match = pattern.search(line)
        if match:
            leading_space, index, space_after_index, old_value, comment_part, param_name, suffix = match.groups()
            
            # Check if this parameter should be updated
            for warsaw_param, restrict_param in mapping.items():
                if param_name == restrict_param and warsaw_param in parameters:
                    # Replace only the value while keeping the index and rest of the line intact
                    updated_line = f"{leading_space}{index}{space_after_index}{parameters[warsaw_param]}{comment_part}{param_name}{suffix}\n"
                    updated_lines.append(updated_line)
                    break
            else:
                # If no mapping found, keep original line
                updated_lines.append(line + '\n')
        else:
            # For lines without parameters, keep unchanged
            updated_lines.append(line + '\n')
    
    # Write the updated content back to the file
    with open(restrict_file_in + '.updated', 'w') as file:
        file.writelines(updated_lines)
    
    return restrict_file_in + '.updated'

In [10]:
"""
Mapping from Warsaw_rosetta to SMEFTsim friendly input card
"""
variable_map = {
    'cll1111' : 'cllRe1111',
    'cll1122' : 'cllRe1122',
    'cll1221' : 'cllRe1221',
    'cll1133' : 'cllRe1133',
    'cll1331' : 'cllRe1331',
    'cll2332' : 'cllRe2332',
    'cle1111' : 'cleRe1111',
    'cle1122' : 'cleRe1122',
    'cle2211' : 'cleRe2211',
    'cle1133' : 'cleRe1133',
    'cle3311' : 'cleRe3311',
    'cee1111' : 'ceeRe1111',
    'cee1122' : 'cllRe1122',
    'cee1133' : 'cllRe1133',
    'cHbx' : 'cHbox',
    'cHD' : 'cHDD',
    'cH' : 'cH',
    'cHG' : 'cHG',
    'cHW' : 'cHW',
    'cHB' : 'cHB',
    'cHWB' : 'cHWB',
    'tcHG' : 'cHGtil',
    'tcHW' : 'cHWtil',
    'tcHB' : 'cHBtil',
    'tcHWB' : 'cHWBtil',
    'cW' : 'cW',
    'cG' : 'cG',
    'tcW' : 'cWtil',
    'tcG' : 'cGtil',
    'RcdB1x1' : 'cdBRe11',
    'RcdB1x2' : 'cdBRe12',
    'RcdB1x3' : 'cdBRe13',
    'RcdB2x1' : 'cdBRe21',
    'RcdB2x2' : 'cdBRe22',
    'RcdB2x3' : 'cdBRe23',
    'RcdB3x1' : 'cdBRe31',
    'RcdB3x2' : 'cdBRe32',
    'RcdB3x3' : 'cdBRe33',
    'IcdB1x1' : 'cdBIm11',
    'IcdB1x2' : 'cdBIm12',
    'IcdB1x3' : 'cdBIm13',
    'IcdB2x1' : 'cdBIm21',
    'IcdB2x2' : 'cdBIm22',
    'IcdB2x3' : 'cdBIm23',
    'IcdB3x1' : 'cdBIm31',
    'IcdB3x2' : 'cdBIm32',
    'IcdB3x3' : 'cdBIm33',
    'RcdG1x1' : 'cdGRe11',
    'RcdG1x2' : 'cdGRe12',
    'RcdG1x3' : 'cdGRe13',
    'RcdG2x1' : 'cdGRe21',
    'RcdG2x2' : 'cdGRe22',
    'RcdG2x3' : 'cdGRe23',
    'RcdG3x1' : 'cdGRe31',
    'RcdG3x2' : 'cdGRe32',
    'RcdG3x3' : 'cdGRe33',
    'IcdG1x1' : 'cdGIm11',
    'IcdG1x2' : 'cdGIm12',
    'IcdG1x3' : 'cdGIm13',
    'IcdG2x1' : 'cdGIm21',
    'IcdG2x2' : 'cdGIm22',
    'IcdG2x3' : 'cdGIm23',
    'IcdG3x1' : 'cdGIm31',
    'IcdG3x2' : 'cdGIm32',
    'IcdG3x3' : 'cdGIm33',
    'RcdH1x1' : 'cdHRe11',
    'RcdH1x2' : 'cdHRe12',
    'RcdH1x3' : 'cdHRe13',
    'RcdH2x1' : 'cdHRe21',
    'RcdH2x2' : 'cdHRe22',
    'RcdH2x3' : 'cdHRe23',
    'RcdH3x1' : 'cdHRe31',
    'RcdH3x2' : 'cdHRe32',
    'RcdH3x3' : 'cdHRe33',
    'IcdH1x1' : 'cdHIm11',
    'IcdH1x2' : 'cdHIm12',
    'IcdH1x3' : 'cdHIm13',
    'IcdH2x1' : 'cdHIm21',
    'IcdH2x2' : 'cdHIm22',
    'IcdH2x3' : 'cdHIm23',
    'IcdH3x1' : 'cdHIm31',
    'IcdH3x2' : 'cdHIm32',
    'IcdH3x3' : 'cdHIm33',
    'RcdW1x1' : 'cdWRe11',
    'RcdW1x2' : 'cdWRe12',
    'RcdW1x3' : 'cdWRe13',
    'RcdW2x1' : 'cdWRe21',
    'RcdW2x2' : 'cdWRe22',
    'RcdW2x3' : 'cdWRe23',
    'RcdW3x1' : 'cdWRe31',
    'RcdW3x2' : 'cdWRe32',
    'RcdW3x3' : 'cdWRe33',
    'IcdW1x1' : 'cdWIm11',
    'IcdW1x2' : 'cdWIm12',
    'IcdW1x3' : 'cdWIm13',
    'IcdW2x1' : 'cdWIm21',
    'IcdW2x2' : 'cdWIm22',
    'IcdW2x3' : 'cdWIm23',
    'IcdW3x1' : 'cdWIm31',
    'IcdW3x2' : 'cdWIm32',
    'IcdW3x3' : 'cdWIm33',
    'RceB1x1' : 'ceBRe11',
    'RceB1x2' : 'ceBRe12',
    'RceB1x3' : 'ceBRe13',
    'RceB2x1' : 'ceBRe21',
    'RceB2x2' : 'ceBRe22',
    'RceB2x3' : 'ceBRe23',
    'RceB3x1' : 'ceBRe31',
    'RceB3x2' : 'ceBRe32',
    'RceB3x3' : 'ceBRe33',
    'IceB1x1' : 'ceBIm11',
    'IceB1x2' : 'ceBIm12',
    'IceB1x3' : 'ceBIm13',
    'IceB2x1' : 'ceBIm21',
    'IceB2x2' : 'ceBIm22',
    'IceB2x3' : 'ceBIm23',
    'IceB3x1' : 'ceBIm31',
    'IceB3x2' : 'ceBIm32',
    'IceB3x3' : 'ceBIm33',
    'RceH1x1' : 'ceHRe11',
    'RceH1x2' : 'ceHRe12',
    'RceH1x3' : 'ceHRe13',
    'RceH2x1' : 'ceHRe21',
    'RceH2x2' : 'ceHRe22',
    'RceH2x3' : 'ceHRe23',
    'RceH3x1' : 'ceHRe31',
    'RceH3x2' : 'ceHRe32',
    'RceH3x3' : 'ceHRe33',
    'IceH1x1' : 'ceHIm11',
    'IceH1x2' : 'ceHIm12',
    'IceH1x3' : 'ceHIm13',
    'IceH2x1' : 'ceHIm21',
    'IceH2x2' : 'ceHIm22',
    'IceH2x3' : 'ceHIm23',
    'IceH3x1' : 'ceHIm31',
    'IceH3x2' : 'ceHIm32',
    'IceH3x3' : 'ceHIm33',
    'RceW1x1' : 'ceWRe11',
    'RceW1x2' : 'ceWRe12',
    'RceW1x3' : 'ceWRe13',
    'RceW2x1' : 'ceWRe21',
    'RceW2x2' : 'ceWRe22',
    'RceW2x3' : 'ceWRe23',
    'RceW3x1' : 'ceWRe31',
    'RceW3x2' : 'ceWRe32',
    'RceW3x3' : 'ceWRe33',
    'IceW1x1' : 'ceWIm11',
    'IceW1x2' : 'ceWIm12',
    'IceW1x3' : 'ceWIm13',
    'IceW2x1' : 'ceWIm21',
    'IceW2x2' : 'ceWIm22',
    'IceW2x3' : 'ceWIm23',
    'IceW3x1' : 'ceWIm31',
    'IceW3x2' : 'ceWIm32',
    'IceW3x3' : 'ceWIm33',
    'RcH1l1x1' : 'cHl1Re11',
    'RcH1l1x2' : 'cHl1Re12',
    'RcH1l1x3' : 'cHl1Re13',
    'RcH1l2x2' : 'cHl1Re22',
    'RcH1l2x3' : 'cHl1Re23',
    'RcH1l3x3' : 'cHl1Re33',
    'IcH1l1x2' : 'cHl1Im12',
    'IcH1l1x3' : 'cHl1Im13',
    'IcH1l2x3' : 'cHl1Im23',
    'RcH1q1x1' : 'cHq1Re11',
    'RcH1q1x2' : 'cHq1Re12',
    'RcH1q1x3' : 'cHq1Re13',
    'RcH1q2x2' : 'cHq1Re22',
    'RcH1q2x3' : 'cHq1Re23',
    'RcH1q3x3' : 'cHq1Re33',
    'IcH1q1x2' : 'cHq1Im12',
    'IcH1q1x3' : 'cHq1Im13',
    'IcH1q2x3' : 'cHq1Im23',
    'RcH3l1x1' : 'cHl3Re11',
    'RcH3l1x2' : 'cHl3Re12',
    'RcH3l1x3' : 'cHl3Re13',
    'RcH3l2x2' : 'cHl3Re22',
    'RcH3l2x3' : 'cHl3Re23',
    'RcH3l3x3' : 'cHl3Re33',
    'IcH3l1x2' : 'cHl3Im12',
    'IcH3l1x3' : 'cHl3Im13',
    'IcH3l2x3' : 'cHl3Im23',
    'RcH3q1x1' : 'cHq3Re11',
    'RcH3q1x2' : 'cHq3Re12',
    'RcH3q1x3' : 'cHq3Re13',
    'RcH3q2x2' : 'cHq3Re22',
    'RcH3q2x3' : 'cHq3Re23',
    'RcH3q3x3' : 'cHq3Re33',
    'IcH3q1x2' : 'cHq3Im12',
    'IcH3q1x3' : 'cHq3Im13',
    'IcH3q2x3' : 'cHq3Im23',
    'RcHd1x1' : 'cHdRe11',
    'RcHd1x2' : 'cHdRe12',
    'RcHd1x3' : 'cHdRe13',
    'RcHd2x2' : 'cHdRe22',
    'RcHd2x3' : 'cHdRe23',
    'RcHd3x3' : 'cHdRe33',
    'IcHd1x2' : 'cHdIm12',
    'IcHd1x3' : 'cHdIm13',
    'IcHd2x3' : 'cHdIm23',
    'RcHe1x1' : 'cHeRe11',
    'RcHe1x2' : 'cHeRe12',
    'RcHe1x3' : 'cHeRe13',
    'RcHe2x2' : 'cHeRe22',
    'RcHe2x3' : 'cHeRe23',
    'RcHe3x3' : 'cHeRe33',
    'IcHe1x2' : 'cHeIm12',
    'IcHe1x3' : 'cHeIm13',
    'IcHe2x3' : 'cHeIm23',
    'RcHu1x1' : 'cHuRe11',
    'RcHu1x2' : 'cHuRe12',
    'RcHu1x3' : 'cHuRe13',
    'RcHu2x2' : 'cHuRe22',
    'RcHu2x3' : 'cHuRe23',
    'RcHu3x3' : 'cHuRe33',
    'IcHu1x2' : 'cHuIm12',
    'IcHu1x3' : 'cHuIm13',
    'IcHu2x3' : 'cHuIm23',
    'RcHud1x1' : 'cHudRe11',
    'RcHud1x2' : 'cHudRe12',
    'RcHud1x3' : 'cHudRe13',
    'RcHud2x1' : 'cHudRe21',
    'RcHud2x2' : 'cHudRe22',
    'RcHud2x3' : 'cHudRe23',
    'RcHud3x1' : 'cHudRe31',
    'RcHud3x2' : 'cHudRe32',
    'RcHud3x3' : 'cHudRe33',
    'IcHud1x1' : 'cHudIm11',
    'IcHud1x2' : 'cHudIm12',
    'IcHud1x3' : 'cHudIm13',
    'IcHud2x1' : 'cHudIm21',
    'IcHud2x2' : 'cHudIm22',
    'IcHud2x3' : 'cHudIm23',
    'IcHud3x1' : 'cHudIm31',
    'IcHud3x2' : 'cHudIm32',
    'IcHud3x3' : 'cHudIm33',
    'RcuB1x1' : 'cuBRe11',
    'RcuB1x2' : 'cuBRe12',
    'RcuB1x3' : 'cuBRe13',
    'RcuB2x1' : 'cuBRe21',
    'RcuB2x2' : 'cuBRe22',
    'RcuB2x3' : 'cuBRe23',
    'RcuB3x1' : 'cuBRe31',
    'RcuB3x2' : 'cuBRe32',
    'RcuB3x3' : 'cuBRe33',
    'IcuB1x1' : 'cuBIm11',
    'IcuB1x2' : 'cuBIm12',
    'IcuB1x3' : 'cuBIm13',
    'IcuB2x1' : 'cuBIm21',
    'IcuB2x2' : 'cuBIm22',
    'IcuB2x3' : 'cuBIm23',
    'IcuB3x1' : 'cuBIm31',
    'IcuB3x2' : 'cuBIm32',
    'IcuB3x3' : 'cuBIm33',
    'RcuG1x1' : 'cuGRe11',
    'RcuG1x2' : 'cuGRe12',
    'RcuG1x3' : 'cuGRe13',
    'RcuG2x1' : 'cuGRe21',
    'RcuG2x2' : 'cuGRe22',
    'RcuG2x3' : 'cuGRe23',
    'RcuG3x1' : 'cuGRe31',
    'RcuG3x2' : 'cuGRe32',
    'RcuG3x3' : 'cuGRe33',
    'IcuG1x1' : 'cuGIm11',
    'IcuG1x2' : 'cuGIm12',
    'IcuG1x3' : 'cuGIm13',
    'IcuG2x1' : 'cuGIm21',
    'IcuG2x2' : 'cuGIm22',
    'IcuG2x3' : 'cuGIm23',
    'IcuG3x1' : 'cuGIm31',
    'IcuG3x2' : 'cuGIm32',
    'IcuG3x3' : 'cuGIm33',
    'RcuH1x1' : 'cuHRe11',
    'RcuH1x2' : 'cuHRe12',
    'RcuH1x3' : 'cuHRe13',
    'RcuH2x1' : 'cuHRe21',
    'RcuH2x2' : 'cuHRe22',
    'RcuH2x3' : 'cuHRe23',
    'RcuH3x1' : 'cuHRe31',
    'RcuH3x2' : 'cuHRe32',
    'RcuH3x3' : 'cuHRe33',
    'IcuH1x1' : 'cuHIm11',
    'IcuH1x2' : 'cuHIm12',
    'IcuH1x3' : 'cuHIm13',
    'IcuH2x1' : 'cuHIm21',
    'IcuH2x2' : 'cuHIm22',
    'IcuH2x3' : 'cuHIm23',
    'IcuH3x1' : 'cuHIm31',
    'IcuH3x2' : 'cuHIm32',
    'IcuH3x3' : 'cuHIm33',
    'RcuW1x1' : 'cuWRe11',
    'RcuW1x2' : 'cuWRe12',
    'RcuW1x3' : 'cuWRe13',
    'RcuW2x1' : 'cuWRe21',
    'RcuW2x2' : 'cuWRe22',
    'RcuW2x3' : 'cuWRe23',
    'RcuW3x1' : 'cuWRe31',
    'RcuW3x2' : 'cuWRe32',
    'RcuW3x3' : 'cuWRe33',
    'IcuW1x1' : 'cuWIm11',
    'IcuW1x2' : 'cuWIm12',
    'IcuW1x3' : 'cuWIm13',
    'IcuW2x1' : 'cuWIm21',
    'IcuW2x2' : 'cuWIm22',
    'IcuW2x3' : 'cuWIm23',
    'IcuW3x1' : 'cuWIm31',
    'IcuW3x2' : 'cuWIm32',
    'IcuW3x3' : 'cuWIm33'
}

In [14]:
# Files
"""
warsaw_file : The output from Rosetta
restrict_file : The SMEFTsim friendly input card with all WCs=0
"""
warsaw_file = "/Users/keshavchaudhary/Downloads/Rosetta/warsaw_cW_scaled_65_99.dat"
restrict_file = "/Users/keshavchaudhary/Desktop/Uni_Pisa_PhD/SMEFT_project/restrict_all_zero.dat"

In [16]:
"""
The print statements here were just for debugging purposes and can be commented
"""
# Read parameters from warsaw file
print(f"Reading parameters from {warsaw_file}...")
warsaw_parameters = read_warsaw_parameters(warsaw_file)
print(f"Read {len(warsaw_parameters)} parameters from {warsaw_file}")

# Read and display some lines from restrict file for debugging
print(f"\nSample lines from {restrict_file} (before updating):")
with open(restrict_file, 'r') as f:
    for i, line in enumerate(f):
        if i < 5 and '#' in line:  # Show first 5 parameter lines
            print(f"  {line.strip()}")
        if i > 10:
            break

Reading parameters from /Users/keshavchaudhary/Downloads/Rosetta/warsaw_cW_scaled_65_99.dat...
Found in warsaw file: Mu = +4.80000e-03
Found in warsaw file: Md = +2.30000e-03
Found in warsaw file: Ms = +9.50000e-02
Found in warsaw file: Mc = +1.42000e+00
Found in warsaw file: Mb = +4.70000e+00
Found in warsaw file: Mt = +1.73000e+02
Found in warsaw file: Me = +5.11000e-04
Found in warsaw file: Mmu = +1.05658e-01
Found in warsaw file: Mta = +1.77680e+00
Found in warsaw file: MZ = +9.11876e+01
Found in warsaw file: MW = +8.03850e+01
Found in warsaw file: MH = +1.25000e+02
Found in warsaw file: aEWM1 = +1.27916e+02
Found in warsaw file: Gf = +1.16638e-05
Found in warsaw file: aS = +1.18400e-01
Found in warsaw file: MZ = +9.11876e+01
Found in warsaw file: MH = +1.25000e+02
Found in warsaw file: RVud1x1 = +9.74272e-01
Found in warsaw file: RVus1x2 = +2.25349e-01
Found in warsaw file: RVub1x3 = +1.21581e-03
Found in warsaw file: RVcd2x1 = -2.25208e-01
Found in warsaw file: RVcs2x2 = +9.73440

In [21]:
"""
A new file will be created in the same folder as restrict_file, with the old name appended with "updated"
"""
# Update restrict file with warsaw parameters
output_file = update_restrict_file(restrict_file, warsaw_parameters, variable_map)
print(f"\nUpdated parameters written to {output_file}")

# Print what mappings were applied
print("\nApplied parameter mappings:")
for warsaw_param, restrict_param in variable_map.items():
    if warsaw_param in warsaw_parameters:
        print(f"{warsaw_param} ({warsaw_parameters[warsaw_param]}) → {restrict_param}")
        
# Show some lines from the updated file
print(f"\nSample lines from {output_file} (after updating):")
with open(output_file, 'r') as f:
    for i, line in enumerate(f):
        if i < 5 and '#' in line:  # Show first 5 parameter lines
            print(f"  {line.strip()}")
        if i > 10:
            break


Updated parameters written to /Users/keshavchaudhary/Desktop/Uni_Pisa_PhD/SMEFT_project/restrict_all_zero.dat.updated

Applied parameter mappings:
cll1111 (+0.00000e+00) → cllRe1111
cll1122 (+0.00000e+00) → cllRe1122
cll1221 (+0.00000e+00) → cllRe1221
cll1133 (+0.00000e+00) → cllRe1133
cll1331 (+0.00000e+00) → cllRe1331
cll2332 (+0.00000e+00) → cllRe2332
cle1111 (+0.00000e+00) → cleRe1111
cle1122 (+0.00000e+00) → cleRe1122
cle2211 (+0.00000e+00) → cleRe2211
cle1133 (+0.00000e+00) → cleRe1133
cle3311 (+0.00000e+00) → cleRe3311
cee1111 (+0.00000e+00) → ceeRe1111
cee1122 (+0.00000e+00) → cllRe1122
cee1133 (+0.00000e+00) → cllRe1133
cHbx (+1.97970e+02) → cHbox
cHD (+0.00000e+00) → cHDD
cH (+6.80320e+01) → cH
cHG (+0.00000e+00) → cHG
cHW (-0.00000e+00) → cHW
cHB (+0.00000e+00) → cHB
cHWB (-0.00000e+00) → cHWB
tcHG (+0.00000e+00) → cHGtil
tcHW (-0.00000e+00) → cHWtil
tcHB (+0.00000e+00) → cHBtil
tcHWB (-0.00000e+00) → cHWBtil
cW (+0.00000e+00) → cW
cG (+0.00000e+00) → cG
tcW (+0.00000e+00) 